# Flowers Image Classification with TensorFlow on Cloud ML Engine TPU

This notebook demonstrates how to do image classification from scratch on a flowers dataset using the Estimator API. Unlike [flowers_fromscratch.ipynb](the flowers_fromscratch notebook), here we do it on a TPU.

Therefore, this will work only if you have quota for TPUs (not in Qwiklabs). It will cost about $3 if you want to try it out.

In [ ]:
%bash
pip install apache-beam[gcp]

After doing a pip install, click on Reset Session so that the Python environment picks up the new package

In [1]:
import os
PROJECT = 'cloud-training-demos' # REPLACE WITH YOUR PROJECT ID
BUCKET = 'cloud-training-demos-ml' # REPLACE WITH YOUR BUCKET NAME
REGION = 'us-central1' # REPLACE WITH YOUR BUCKET REGION e.g. us-central1

# do not change these
os.environ['PROJECT'] = PROJECT
os.environ['BUCKET'] = BUCKET
os.environ['REGION'] = REGION
os.environ['TFVERSION'] = '1.8'  # Tensorflow version

In [2]:
%bash
gcloud config set project $PROJECT
gcloud config set compute/region $REGION

Updated property [core/project].
Updated property [compute/region].


## Preprocess JPEG images to TF Records

While using a GPU, it is okay to read the JPEGS directly from our input_fn. However, TPUs are too fast and it will be very wasteful to have the TPUs wait on I/O. Therefore, we'll preprocess the JPEGs into TF Records.

This runs on Cloud Dataflow and will take <b> 15-20 minutes </b>

In [3]:
%bash
gsutil cat gs://cloud-ml-data/img/flower_photos/train_set.csv  | sed 's/,/ /g' | awk '{print $2}' | sort | uniq > /tmp/labels.txt

In [ ]:
%bash
export PYTHONPATH=${PYTHONPATH}:${PWD}/flowersmodeltpu
gsutil -m rm -rf gs://${BUCKET}/tpu/flowers/data
python -m trainer.preprocess \
       --train_csv gs://cloud-ml-data/img/flower_photos/train_set.csv \
       --validation_csv gs://cloud-ml-data/img/flower_photos/eval_set.csv \
       --labels_file /tmp/labels.txt \
       --project_id $PROJECT \
       --output_dir gs://${BUCKET}/tpu/flowers/data

In [6]:
%bash
gsutil ls gs://${BUCKET}/tpu/flowers/data/

gs://cloud-training-demos-ml/tpu/flowers/data/train-00000-of-00008
gs://cloud-training-demos-ml/tpu/flowers/data/train-00001-of-00008
gs://cloud-training-demos-ml/tpu/flowers/data/train-00002-of-00008
gs://cloud-training-demos-ml/tpu/flowers/data/train-00003-of-00008
gs://cloud-training-demos-ml/tpu/flowers/data/train-00004-of-00008
gs://cloud-training-demos-ml/tpu/flowers/data/train-00005-of-00008
gs://cloud-training-demos-ml/tpu/flowers/data/train-00006-of-00008
gs://cloud-training-demos-ml/tpu/flowers/data/train-00007-of-00008
gs://cloud-training-demos-ml/tpu/flowers/data/validation-00000-of-00003
gs://cloud-training-demos-ml/tpu/flowers/data/validation-00001-of-00003
gs://cloud-training-demos-ml/tpu/flowers/data/validation-00002-of-00003
gs://cloud-training-demos-ml/tpu/flowers/data/tmp/


## Run as a Python module

Let's first run it locally for a short while to test the code works. Note the --model parameter

In [ ]:
%bash
rm -rf flowers_trained
gcloud ml-engine local train \
   --module-name=flowersmodeltpu.task \
   --package-path=${PWD}/flowersmodeltpu \
   -- \
   --output_dir=${PWD}/flowers_trained \
   --train_steps=5 \
   --learning_rate=0.01 \
   --batch_size=2 \
   --train_data_path=gs://${BUCKET}/tpu/flowers/data/train-00000-* \
   --eval_data_path=gs://${BUCKET}/tpu/flowers/data/validation-00000-*

Now, let's do it on ML Engine. Note the --model parameter

In [ ]:
%bash
OUTDIR=gs://${BUCKET}/flowers/trained_${MODEL_TYPE}
JOBNAME=flowers_${MODEL_TYPE}_$(date -u +%y%m%d_%H%M%S)
echo $OUTDIR $REGION $JOBNAME
gsutil -m rm -rf $OUTDIR
gcloud ml-engine jobs submit training $JOBNAME \
   --region=$REGION \
   --module-name=flowersmodel.task \
   --package-path=${PWD}/flowersmodel \
   --job-dir=$OUTDIR \
   --staging-bucket=gs://$BUCKET \
   --scale-tier=BASIC_GPU \
   --runtime-version=$TFVERSION \
   -- \
   --output_dir=$OUTDIR \
   --train_steps=1000 \
   --learning_rate=0.01 \
   --batch_size=40 \
   --model=$MODEL_TYPE \
   --augment \
   --batch_norm \
   --train_data_path=gs://cloud-ml-data/img/flower_photos/train_set.csv \
   --eval_data_path=gs://cloud-ml-data/img/flower_photos/eval_set.csv

## Monitoring training with TensorBoard

Use this cell to launch tensorboard

In [ ]:
from google.datalab.ml import TensorBoard
TensorBoard().start('gs://{}/flowers/trained_{}'.format(BUCKET, MODEL_TYPE))

In [ ]:
for pid in TensorBoard.list()['pid']:
  TensorBoard().stop(pid)
  print 'Stopped TensorBoard with pid {}'.format(pid)

Here are my results:

Model | Accuracy | Time taken | Run time parameters
--- | :---: | ---
cnn with batch-norm | 0.582 | 47 min | 1000 steps, LR=0.01, Batch=40
as above, plus augment | 0.615 | 3 hr | 5000 steps, LR=0.01, Batch=40

## Deploying and predicting with model

Deploy the model:

In [ ]:
%bash
MODEL_NAME="flowers"
MODEL_VERSION=${MODEL_TYPE}
MODEL_LOCATION=$(gsutil ls gs://${BUCKET}/flowers/trained_${MODEL_TYPE}/export/exporter | tail -1)
echo "Deleting and deploying $MODEL_NAME $MODEL_VERSION from $MODEL_LOCATION ... this will take a few minutes"
#gcloud ml-engine versions delete --quiet ${MODEL_VERSION} --model ${MODEL_NAME}
#gcloud ml-engine models delete ${MODEL_NAME}
gcloud ml-engine models create ${MODEL_NAME} --regions $REGION
gcloud ml-engine versions create ${MODEL_VERSION} --model ${MODEL_NAME} --origin ${MODEL_LOCATION} --runtime-version=$TFVERSION

To predict with the model, let's take one of the example images that is available on Google Cloud Storage <img src="http://storage.googleapis.com/cloud-ml-data/img/flower_photos/sunflowers/1022552002_2b93faf9e7_n.jpg" />

The online prediction service expects images to be base64 encoded as described [here](https://cloud.google.com/ml-engine/docs/tensorflow/online-predict#binary_data_in_prediction_input).

In [ ]:
%%bash
IMAGE_URL=gs://cloud-ml-data/img/flower_photos/sunflowers/1022552002_2b93faf9e7_n.jpg

# Copy the image to local disk.
gsutil cp $IMAGE_URL flower.jpg

# Base64 encode and create request message in json format.
python -c 'import base64, sys, json; img = base64.b64encode(open("flower.jpg", "rb").read()); print json.dumps({"image_bytes":{"b64": img}})' &> request.json

Send it to the prediction service

In [ ]:
%%bash
gcloud ml-engine predict \
  --model=flowers \
  --version=${MODEL_TYPE} \
  --json-instances=./request.json

<pre>
# Copyright 2017 Google Inc. All Rights Reserved.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#      http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
</pre>